In [1]:
import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot as plt
import os
from dataset import *
from vladder import VLadder
from visualize import *

In [2]:
dataset = MnistDataset()
model = VLadder(dataset, name='', reg='kl', batch_size=100, restart=False)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /mnt/c/Users/Michael/Documents/Variational-Ladder-Autoencoder/dataset/MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /mnt/c/Users/Michael/Documents/Variational-Ladder-Autoencoder/dataset/MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /mnt/c/Users/Michael/Documents/Variational-Ladder-Autoencoder/dataset/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /mnt/c/Users/Michael/Documents/Variational-Ladder-Autoencoder/dataset/MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
INFO:tensorflow:Resto

In [3]:
class ConditionalManifoldSampleVisualizer(Visualizer):
    def __init__(self, network, dataset):
        Visualizer.__init__(self, network)
        self.dataset = dataset
        self.name = "conditional_manifold_samples"
        
    def visualize(self, layers, num_rows=4, use_gui=False):
        if use_gui and self.fig is None:
            self.fig, self.ax = plt.subplots(1, len(layers))
        self.latent_code = self.network.random_latent_code()
        
        canvas_list = []
        for i, layer in enumerate(layers):
            samples = np.zeros([num_rows*num_rows]+self.dataset.data_dims)
            samples_ptr = 0
            layer_latent_code_x = np.tile(np.reshape(np.linspace(-2.0, 2.0, num=num_rows), (1, num_rows)), (num_rows, 1))
            layer_latent_code_y = layer_latent_code_x.transpose()
            layer_latent_code = np.reshape(np.stack([layer_latent_code_x, layer_latent_code_y], axis=-1), (-1, 2))
            while samples_ptr < num_rows * num_rows:
                new_samples = self.network.generate_conditional_manifold_samples(layer, layer_latent_code[samples_ptr:samples_ptr+8], self.latent_code)
                next_ptr = samples_ptr + 8
                if next_ptr > num_rows * num_rows:
                    next_ptr = num_rows * num_rows

                samples[samples_ptr:next_ptr] = new_samples
                samples_ptr = next_ptr

            # Plot the samples in a grid
            if samples is not None:
                samples = self.dataset.display(samples)
                width = samples.shape[1]
                height = samples.shape[2]
                channel = samples.shape[3]
                canvas = np.zeros((width * num_rows, height * num_rows, channel))
                for img_index1 in range(num_rows):
                    for img_index2 in range(num_rows):
                        canvas[img_index1 * width:(img_index1 + 1) * width,
                            img_index2 * height:(img_index2 + 1) * height, :] = \
                            samples[img_index1 * num_rows + img_index2, :, :, :]
                if use_gui:
                    self.ax[i].cla()
                    if channel == 1:
                        self.ax[i].imshow(canvas[:, :, 0], cmap=plt.get_cmap('Greys'))
                    else:
                        self.ax[i].imshow(canvas)
                    self.ax[i].xaxis.set_visible(False)
                    self.ax[i].yaxis.set_visible(False)
                if i != 0:
                    if canvas.shape[-1] == 1:
                        canvas_list.append(np.zeros((width * num_rows, 20, channel)))
                    else:
                        canvas_list.append(np.ones((width * num_rows, 20, channel)))
                canvas_list.append(canvas)
            else:
                print("Warning: no samples generated during visualization")
        # np.save('samples', canvas)
        canvas = np.concatenate(canvas_list, axis=1)
        self.arr_to_file(canvas)
        if use_gui:
            self.fig.suptitle('Conditional Samples for %s' % self.network.name)
            plt.draw()
            plt.pause(0.01)

In [9]:
sampler_visualizer = ConditionalManifoldSampleVisualizer(model, dataset)

In [10]:
layers = [layer for layer in model.random_latent_code()]
layers.sort()
sampler_visualizer.visualize(layers, 32)